<a href="https://colab.research.google.com/github/vanessajen/FlatLay-Generator/blob/main/FlatLay_Generator_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- Bước 0: Import thư viện cần thiết ---
!pip install diffusers transformers accelerate controlnet_aux --upgrade

# --- Bước 1: Set môi trường ---
import os
os.environ["DIFFUSERS_FLAX_AVAILABLE"] = "0"

# --- Bước 2: Import thư viện Python ---
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from controlnet_aux import CannyDetector
from PIL import Image
import numpy as np

# --- Bước 3: Load model ControlNet ---
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16
)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    safety_checker=None,
    torch_dtype=torch.float16
)

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
# pipe.enable_xformers_memory_efficient_attention()  <-- Bỏ dòng này đi nha chị
pipe.to("cuda")

# --- Bước 4: Khai báo Canny Detector ---
canny = CannyDetector()

# --- Bước 5: Hàm xử lý ảnh ---
def process_flatlay(input_image_path, prompt, output_path):
    image = Image.open(input_image_path)
    image = image.convert("RGB")
    image = image.resize((512, 512))

    # Tính toán canny map
    canny_image = canny(image)

    # Generate ảnh
    generated_image = pipe(
        prompt,
        num_inference_steps=30,
        guidance_scale=9.0,
        image=canny_image
    ).images[0]

    # Lưu ảnh
    generated_image.save(output_path)

# --- Bước 6: Demo ---
input_image_path = "/content/drive/MyDrive/FlatLay_Input/sample.jpg"  # Chị đổi path cho đúng nha
output_image_path = "/content/drive/MyDrive/FlatLay_Demo_Output/output_sample.jpg"
prompt = "flat lay minimal pastel background, photography, highly detailed, white background"

process_flatlay(input_image_path, prompt, output_image_path)

print("🌿✨ Done!")
